In [31]:
import pandas as pd
import numpy as np
from collections import Counter

#Reading data

df_training = pd.read_csv("Training_dataset_Original.csv")
df_eval = pd.read_csv("Evaluation_dataset.csv")
df_leader = pd.read_csv("Leaderboard_dataset.csv")
df_datadict = pd.read_csv("Data_Dictionary.csv")

C:\Users\Utkarsh\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#replace all missing values with mean of existing values in training set
#same can be done for testing set but data leakage can occur




In [ ]:
#general informations

n_people = df_training.shape[0]
n_features = df_training.shape[1]-1
print("Total number of people: {}".format(n_people))
print("Number of features: {}".format(n_features))

In [ ]:
#preparing data

# Extract feature (X) and target (y) columns
feature_cols = list(df_training.columns[:-1])  # all columns but last are features
target_col = df_training.columns[-1]  # last column is the target/label
print("Feature column(s):-\n{}".format(feature_cols))
print( "Target column: {}".format(target_col))

X_all = df_training[feature_cols]  # feature values for all people
y_all = df_training[target_col]  # corresponding targets/labels
print ("\nFeature values:-")
print(X_all.head())  # print the first 5 rows


In [ ]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)

        outX = outX.join(col_data)  # collect columns in output dataframe

    return outX

X_all = preprocess_features(X_all)
print("Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

In [ ]:
# preparing number of training and test samples
num_all = df_training.shape[0]  # same as len(df_training)
num_train = 0.9 * num_all # about 90% of the data
num_test = num_all - num_train

# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn import cross_validation

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_all, y_all, test_size=num_test)
print("Training set: {} samples".format(X_train.shape[0]))
print ("Test set: {} samples".format(X_test.shape[0]))
# Note: If you need a validation set, extract it from within training data

In [ ]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print ("Done!\nTraining time (secs): {:.3f}".format(end - start))

from sklearn import ensemble
clf = ensemble.GradientBoostingClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train) 

In [ ]:
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print( "Done!\nPrediction time (secs): {:.3f}".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print ("F1 score for training set: {}".format(train_f1_score))

In [ ]:
# Predict on test data
print("F1 score for test set: {}".format(predict_labels(clf, X_test, y_test)))


#Now just by changing the classifier we can compare the f1 scores

In [ ]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

train_predict(clf, X_train.sample(n=200, random_state=200), y_train.sample(n=200, random_state=200), X_test, y_test)
train_predict(clf, X_train.sample(n=100, random_state=100), y_train.sample(n=100, random_state=100), X_test, y_test)
# Note: Keep the test set constant

In [ ]:
from sklearn import naive_bayes

mnb = naive_bayes.MultinomialNB()
train_predict(mnb, X_train, y_train, X_test, y_test)
train_predict(mnb, X_train.sample(n=200, random_state=201), y_train.sample(n=200, random_state=201), X_test, y_test)
train_predict(mnb, X_train.sample(n=100, random_state=101), y_train.sample(n=100, random_state=101), X_test, y_test)

In [ ]:
from sklearn import tree

dtc = tree.DecisionTreeClassifier()
train_predict(dtc, X_train, y_train, X_test, y_test)
train_predict(dtc, X_train.sample(n=200, random_state=202), y_train.sample(n=200, random_state=202), X_test, y_test)
train_predict(dtc, X_train.sample(n=100, random_state=102), y_train.sample(n=100, random_state=102), X_test, y_test)

In [ ]:
#choosing the best model

from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

parameters = {'max_depth': (1, 2, 3, 4, 5, 6, 7, 8, 9, 10), 'n_estimators': (100, 125, 150, 500)}
sss = cross_validation.StratifiedShuffleSplit(y_train, test_size=num_test)
gs = GridSearchCV(estimator=clf, n_jobs=-1, scoring=make_scorer(f1_score, pos_label='yes'), param_grid=parameters,
                  cv=sss)
gs.fit(X_train, y_train)
best_estimator = gs.best_estimator_
print("best estimator:\n{}".format(best_estimator))
print ('')
print("best parameter:\n{}".format(gs.best_params_))
print('')
print("F1 score:\n{}".format(f1_score(y_test, best_estimator.predict(X_test), pos_label='yes')))
